<a href="https://colab.research.google.com/github/mamintoosi/Text-Mining/blob/master/Pytorch_(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

import numpy as np

In [ ]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [ ]:
print (chars)
print (int2char)
print (char2int)

{'f', 'a', 'g', 'i', 'u', 'm', 'v', 'c', 'h', 'o', 'y', 'r', ' ', 'w', 'n', 'e', 'd'}
{0: 'f', 1: 'a', 2: 'g', 3: 'i', 4: 'u', 5: 'm', 6: 'v', 7: 'c', 8: 'h', 9: 'o', 10: 'y', 11: 'r', 12: ' ', 13: 'w', 14: 'n', 15: 'e', 16: 'd'}
{'f': 0, 'a': 1, 'g': 2, 'i': 3, 'u': 4, 'm': 5, 'v': 6, 'c': 7, 'h': 8, 'o': 9, 'y': 10, 'r': 11, ' ': 12, 'w': 13, 'n': 14, 'e': 15, 'd': 16}


In [ ]:
# Finding the length of the longest string in our data
maxlen = len(max(text, key=len))

# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of
# the sentence matches the length of the longest sentence
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '

In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
  input_seq.append(text[i][:-1])

    # Remove first character for target sequence
  target_seq.append(text[i][1:])
  print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]


In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [ ]:
# Input shape --> (Batch Size, Sequence Length, One-Hot Encoding Size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [ ]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):

        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [ ]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly

    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.3797
Epoch: 20/100............. Loss: 2.0657
Epoch: 30/100............. Loss: 1.6512
Epoch: 40/100............. Loss: 1.2376
Epoch: 50/100............. Loss: 0.8874
Epoch: 60/100............. Loss: 0.6297
Epoch: 70/100............. Loss: 0.4469
Epoch: 80/100............. Loss: 0.3251
Epoch: 90/100............. Loss: 0.2456
Epoch: 100/100............. Loss: 0.1911


In [ ]:
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)

    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [ ]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
sample(model, 15, 'good')

'good i am fine '

In [ ]:
sample(model, 15, 'hey')

'hey how are you'

In [ ]:
sample(model, 15, 'have')

'have a nice day'

---

---

---

---

---

# Conference

A novel recurrent neural network based of NCP function for solving convex quadratic programming problems

* Sohrab Effati

* The 2nd international Conference on Control and Optimization with Industrial Applications (2008-06-02)

A RECURRENT NEURAL NETWORK FOR SOLVING NONCONVEX NONLINEAR OPTIMIZATION PROBLEM

* Mohammad Moghaddas - Sohrab Effati

* The 44 th Annual Iranian Mathematics Conference Ferdowsi University of Mashhad, (27-30 August 2013)

An projection recurrent neural network model to solve the fuzzy shortest path problem

* Mohammad Eshaghnezhad - Freydoon Rahbarnia - Sohrab Effati

* هشتمین سمینار آمار و احتمال فازی (2018-05-09)

A Projection RNN For Non-Convex Optimization Problems

* Amin Mansoori - Sohrab Effati - Mohammad Eshaghnezhad

* The Third National Seminar on Control and Optimization (2019-11-13)

---

A recurrent neural network-based method for training probabilistic Support Vector Machine

* Hadi Sadoghi Yazdi ، Sohrab Effati

* International Journal of Signal and Imaging Systems Engineering-IJSISE (2009 November)

A novel recurrent nonlinear neural network for solving quadratic programming problems

* Sohrab Effati

* Applied Mathematical Modelling (2011 January)

A Novel Recurrent Neural Network for Solving Mlcps and its Application to Linear and Quadratic Programming

* Sohrab Effati

* Asia-Pacific Journal of Operational Research (2011 November)

An efficient recurrent neural network model for solving fuzzy non-linear programming problems

* Amin Mansoori ، Sohrab Effati ، Mohammad Eshaghnezhad

* Applied Intelligence (2016 January)

Recurrent Neural Network Model: A New Strategy to Solve Fuzzy Matrix Games

* Amin Mansoori ، Mohammad Eshaghnezhad ، Sohrab Effati

* IEEE Transactions on Neural Networks and Learning Systems (2018 December)

Parametric NCP-Based Recurrent Neural Network Model: A New Strategy to Solve Fuzzy Nonconvex Optimization Problems

* Amin Mansoori ، Sohrab Effati

* IEEE Transactions on Systems, Man, and Cybernetics: Systems (2019 January)

Projection Recurrent Neural Network Model: A New Strategy to Solve Maximum Flow Problem

* Mohammad Eshaghnezhad ، Sohrab Effati ، Freydoon Rahbarnia

* IEEE Transactions on Circuits and Systems Part II: Express Briefs (2020 January)

A compact MLCP-based projection recurrent neural network model to solve shortest path problem

* Mohammad Eshaghnezhad ، Sohrab Effati ، Amin Mansoori

* Journal of Experimental and Theoretical Artificial Intelligence (2022 April)

---

**Abstract:**
This paper aims to investigate the fuzzy constrained matrix game -MG- problems using the concepts of recurrent neural networks -RNNs-. To the best of our knowledge, this paper is the first in attempting to find a solution for fuzzy game problems using RNN models. For this purpose, a fuzzy game problem is reformulated into a weighting problem. Then, the Karush–Kuhn–Tucker -KKT- optimality conditions are provided for the weighting problem. The KKT conditions are used to propose the RNN model. Moreover, the Lyapunov stability and the global convergence of the RNN model are also confirmed. Finally, three illustrative examples are presented to demonstrate the effectiveness of this approach. The obtained results are compared with the results obtained by the previous approaches for solving fuzzy constrained MG.

**چکیده:**
هدف این مقاله بررسی مسائل بازی ماتریس محدود فازی با استفاده از مفاهیم شبکه های عصبی بازگشتی است. با بهترین دانش ما، این مقاله اولین مقاله ای است که در تلاش برای یافتن راه حلی برای مسائل بازی فازی با استفاده از مدل های شبکه عصبی بازگشتی است. برای این منظور، یک مسئله بازی فازی به یک مسئله وزنی مجدداً فرموله می شود. سپس شرایط بهینه کاروش-کوهن-تاکر برای مسئله وزن دهی ارائه می شود. شرایط کاروش-کوهن-تاکر برای پیشنهاد مدل شبکه عصبی بازگشتی استفاده می شود. علاوه بر این، پایداری لیاپانوف و همگرایی جهانی مدل شبکه عصبی بازگشتی نیز تایید شده است. در نهایت، سه مثال گویا برای نشان دادن اثربخشی این رویکرد ارائه شده است. نتایج به‌دست‌آمده با نتایج به‌دست‌آمده از روش‌های قبلی برای حل MG مقید فازی مقایسه می‌شود